In [1]:
import DeepMIMO
import numpy as np
import math

# Load the default parameters
parameters = DeepMIMO.default_params()

# Set the main folder containing extracted scenarios
parameters['dataset_folder'] = r'E:\DeepMIMO-codes\DeepMIMO_Dataset_Generation_v1.1\RayTracing Scenarios'

# Set scenario name
parameters['scenario'] = 'O1_3p5B'

#num of multipath
parameters['num_paths'] = 3

#active BS
parameters['active_BS'] = np.array([1,3])

# Define 3 different antennas:
Nix = 8
Niy = 8
Nt = 4
antenna1 = {'shape': np.array([1, Nix, Niy]),#RIS antennas
            'spacing': 0.5,}
antenna2 = {'shape': np.array([1, Nt, Nt]),#BS anteannas
            'spacing': 0.5,}
parameters['bs_antenna'] = [antenna1, antenna2]

# To set the user grid
parameters['user_row_first'] = 500
parameters['user_row_last'] = 600

# UE antennas
Nr = 4
parameters['ue_antenna']['shape'] = np.array([1,Nr,Nr])
parameters['ue_antenna']['spacing'] = 0.5

#non OFDM
parameters['activate_OFDM'] = 0
parameters['OFDM']['bandwidth'] = 0.1 # 100 MHz
parameters['OFDM']['subcarriers'] = 64 # OFDM with 4 subcarriers
parameters['OFDM']['subcarriers_limit'] = 1 # Keep only first 64 subcarriers

# To generate basestation to basestation output variables, set
parameters['enable_BS2BS'] = True

# Generate data
dataset = DeepMIMO.generate_data(parameters)

#print(dataset)
#np.save("./dataset.npy",dataset)


Basestation 1

UE-BS Channels


Generating channels: 100%|██████████| 18281/18281 [00:02<00:00, 7451.16it/s]



BS-BS Channels


Generating channels: 100%|██████████| 2/2 [00:00<00:00, 2004.93it/s]



Basestation 3

UE-BS Channels


Reading ray-tracing: 100%|██████████| 108600/108600 [00:01<00:00, 99337.10it/s]


ToA of some paths of 549 channels with an average total power of 8.29% exceed the useful OFDM symbol duration and are clipped.


Generating channels: 100%|██████████| 18281/18281 [00:01<00:00, 10527.73it/s]



BS-BS Channels


Generating channels: 100%|██████████| 2/2 [00:00<00:00, 2004.45it/s]


In [2]:
# direct link BS-UE channels
dd = np.zeros(((Nr**2)*(Nt**2),18100), dtype=complex)
for count in range(0,18099):
    dlink = (dataset[1]['user']['channel'][count])
    dd[:,count] = dlink.flatten()
print(np.shape(dd))

(256, 18100)


In [3]:
# cascaded channels
GG = np.zeros(((Nr**2)*(Nt**2),18100), dtype=complex)
htlink = (dataset[0]['basestation']['channel'][1])
rr = np.zeros(((Nr**2)*(Nix*Niy),18100), dtype=complex)
for count1 in range(0,18099):
    hrlink = (dataset[0]['user']['channel'][count1])
    rr[:,count1] = (hrlink).flatten()
    Gchannel = np.squeeze(hrlink) @ np.squeeze(htlink).T
    GG[:,count1] = Gchannel.flatten()
print(count1)
print(np.shape(GG))

18098
(256, 18100)


In [4]:
# Overall channels & Noises
Ochannel = GG + dd
# AWGN generation
Pt = 25 #transmit power
Gt = 3
Gr = 3
NF = 5 # noise figure
PG = 10 # processing gain
npower = -204 + 10*math.log(parameters['OFDM']['bandwidth']*10**9,10)+NF-PG #Noise power = -129 dB
npowerl = 10**(0.1*npower)/10**(0.1*(Gt+Gr+Pt))
awgn = np.real((npower/2)**0.5)*(np.random.randn((Nr**2)*(Nt**2),18100)+np.random.randn((Nr**2)*(Nt**2),18100)*1j)
# AWGN channel
awgnchannel = Ochannel + awgn

np.save("./Overallchannel.npy",Ochannel)
np.save("./noisechannel.npy",awgnchannel)
np.save("./receiverlink.npy",rr)
np.save("./directlink.npy",dd)
np.save("./transmitterlink.npy",np.squeeze(htlink))

In [5]:
#print(htlink.shape)
#print(rr.shape)
#print(dd.shape)
print(rr.shape)

(1024, 18100)
